In [1]:
import pandas as pd
import json
import unittest
import csv
from collections import defaultdict
import pandas.testing as pdt

# Optional: Adjust display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
def read_orders_from_csv(file_path):
    """
    Read orders from a CSV file.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        list of dict: List of orders.
    """
    orders = []
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        # Read the CSV file with semicolon as delimiter
        reader = csv.DictReader(csvfile, delimiter=';')
        for row in reader:
            orders.append(row)
    return orders

def read_invoices_from_json(file_path):
    """
    Reads invoices from a JSON file.

    Args:
        file_path (str): Path to the JSON file.

    Returns:
        dict: Dictionary of invoices with orderId as keys.
    """
    with open(file_path, 'r', encoding='utf-8') as jsonfile:
        data = json.load(jsonfile)
        invoices = data.get('data', {}).get('invoices', [])
        invoice_dict = {invoice['orderId']: invoice for invoice in invoices}
    return invoice_dict

Test 1: Distribution of Crate Type per Company

In [4]:
def normalize_company_name(company_name):
    """
    Normalize company names by converting to lowercase and removing common suffixes.

    Args:
        company_name (str): Original company name.

    Returns:
        str: Normalized company name.
    """
    # Convert to lowercase
    name = company_name.lower()
    # Remove common suffixes
    suffixes = [' co', ' co.', ' c.o.', ' inc', ' inc.', ' ltd', ' ltd.', ' gmbh']
    for suffix in suffixes:
        if name.endswith(suffix):
            name = name[:-len(suffix)]
            break
    # Remove any extra whitespace
    name = name.strip()
    return name

def calculate_crate_distribution(orders):
    """
    Calculate the distribution of crate types per company.

    Args:
        orders (list of dict): List of orders, each order is a dict with at least 'company_name' and 'crate_type'.

    Returns:
        dict: Nested dictionary with company names as keys and crate type counts as values.
    """
    distribution = {}

    for order in orders:
        company_name = order.get('company_name')
        crate_type = order.get('crate_type')

        # Skip orders with missing company_name or crate_type
        if not company_name or not crate_type:
            continue

        # Normalize the company name
        normalized_name = normalize_company_name(company_name)

        # Initialize the company entry if it doesn't exist
        if normalized_name not in distribution:
            distribution[normalized_name] = {}

        # Initialize the crate type count if it doesn't exist
        if crate_type not in distribution[normalized_name]:
            distribution[normalized_name][crate_type] = 0

        # Increment the crate type count
        distribution[normalized_name][crate_type] += 1

    return distribution

if __name__ == "__main__":
    # Read orders from 'orders.csv'
    orders = read_orders_from_csv('data/orders.csv')

    # Calculate the crate distribution
    distribution = calculate_crate_distribution(orders)

    # Convert the distribution to a DataFrame
    df = pd.DataFrame.from_dict(distribution, orient='index').fillna(0).astype(int)
    df.index.name = 'Company'
    df.reset_index(inplace=True)

    # Display the DataFrame
    display(df)

,Company,Plastic,Wood,Metal
0,fresh fruits,1,0,0
1,seafood supplier,1,0,1
2,meat packers,1,0,0
3,tropical fruits,1,0,0
4,healthy snacks,1,1,0
5,veggies,0,1,0
6,green veg,0,1,0
7,fresh fruits c.o,0,0,1
8,organic farms,0,0,1


In [5]:
class TestCrateDistribution(unittest.TestCase):

    def test_empty_orders(self):
        """Test with an empty list of orders."""
        orders = []
        expected = {}
        result = calculate_crate_distribution(orders)
        self.assertEqual(result, expected)

    def test_single_order(self):
        """Test with a single order."""
        orders = [{'company_name': 'Fresh Fruits Co', 'crate_type': 'Plastic'}]
        expected = {'fresh fruits': {'Plastic': 1}}
        result = calculate_crate_distribution(orders)
        self.assertEqual(result, expected)

    def test_multiple_orders_same_company(self):
        """Test multiple orders from the same company with different crate types."""
        orders = [
            {'company_name': 'Fresh Fruits Co', 'crate_type': 'Plastic'},
            {'company_name': 'Fresh Fruits c.o.', 'crate_type': 'Metal'},
            {'company_name': 'FRESH FRUITS CO', 'crate_type': 'Plastic'}
        ]
        expected = {'fresh fruits': {'Plastic': 2, 'Metal': 1}}
        result = calculate_crate_distribution(orders)
        self.assertEqual(result, expected)

    def test_multiple_orders_multiple_companies(self):
        """Test multiple orders from different companies."""
        orders = [
            {'company_name': 'Fresh Fruits Co', 'crate_type': 'Plastic'},
            {'company_name': 'Veggies Inc', 'crate_type': 'Wood'},
            {'company_name': 'Fresh Fruits c.o.', 'crate_type': 'Metal'},
            {'company_name': 'Veggies Inc', 'crate_type': 'Wood'},
            {'company_name': 'Fresh Fruits Co', 'crate_type': 'Plastic'},
            {'company_name': 'Veggies Inc', 'crate_type': 'Plastic'}
        ]
        expected = {
            'fresh fruits': {'Plastic': 2, 'Metal': 1},
            'veggies': {'Wood': 2, 'Plastic': 1}
        }
        result = calculate_crate_distribution(orders)
        self.assertEqual(result, expected)

    def test_company_name_normalization(self):
        """Test that company names are normalized correctly."""
        orders = [
            {'company_name': 'Meat Packers Ltd', 'crate_type': 'Plastic'},
            {'company_name': 'Meat Packers', 'crate_type': 'Metal'},
            {'company_name': 'MEAT PACKERS LTD.', 'crate_type': 'Wood'}
        ]
        expected = {'meat packers': {'Plastic': 1, 'Metal': 1, 'Wood': 1}}
        result = calculate_crate_distribution(orders)
        self.assertEqual(result, expected)

    def test_missing_values(self):
        """Test that orders with missing values are skipped."""
        orders = [
            {'company_name': 'Organic Farms', 'crate_type': 'Metal'},
            {'company_name': '', 'crate_type': 'Plastic'},
            {'company_name': 'Organic Farms', 'crate_type': None},
            {'company_name': 'Organic Farms'}
        ]
        expected = {'organic farms': {'Metal': 1}}
        result = calculate_crate_distribution(orders)
        self.assertEqual(result, expected)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.003s

OK


In [6]:
#unittest.main(argv=['first-arg-is-ignored', '-v', 'TestCrateDistribution.test_single_order'], exit=False)

Test 2: DataFrame of Orders with Full Name of the Contact

In [7]:
def extract_contact_full_name(contact_data_str):
    """
    Extracts the full name from the contact_data field.

    Args:
        contact_data_str (str): A string representing contact data in JSON format.

    Returns:
        str: The full name of the contact, or 'John Doe' if not available.
    """
    if not contact_data_str or contact_data_str.strip() == '':
        return 'John Doe'

    try:
        # Handle potential JSON formatting issues
        contact_data_str = contact_data_str.replace("'", '"')
        contact_data = json.loads(contact_data_str)

        # If contact_data is a list, take the first element
        if isinstance(contact_data, list) and contact_data:
            contact_data = contact_data[0]
        elif isinstance(contact_data, dict):
            pass  # contact_data is already a dict
        else:
            return 'John Doe'

        # Extract contact_name and contact_surname
        contact_name = contact_data.get('contact_name', '')
        contact_surname = contact_data.get('contact_surname', '')

        if contact_name and contact_surname:
            full_name = f"{contact_name} {contact_surname}"
        elif contact_name:
            full_name = contact_name
        elif contact_surname:
            full_name = contact_surname
        else:
            full_name = 'John Doe'

        return full_name

    except json.JSONDecodeError:
        # If JSON decoding fails, return 'John Doe'
        return 'John Doe'

def create_contact_dataframe(orders):
    """
    Creates a DataFrame with order_id and contact_full_name columns.

    Args:
        orders (list of dict): List of orders from orders.csv.

    Returns:
        pandas.DataFrame: DataFrame with 'order_id' and 'contact_full_name' columns.
    """
    data = []

    for order in orders:
        order_id = order.get('order_id')
        contact_data_str = order.get('contact_data')
        contact_full_name = extract_contact_full_name(contact_data_str)
        data.append({'order_id': order_id, 'contact_full_name': contact_full_name})

    df = pd.DataFrame(data)
    return df

if __name__ == "__main__":
    # Read orders from 'orders.csv'
    orders = read_orders_from_csv('data/orders.csv')

    # Create the DataFrame with contact full names
    df_1 = create_contact_dataframe(orders)

    # Output the DataFrame
    display(df_1)

,order_id,contact_full_name
0,f47ac10b-58cc-4372-a567-0e02b2c3d479,Curtis Jackson
1,f47ac10b-58cc-4372-a567-0e02b2c3d480,Maria Theresa
2,f47ac10b-58cc-4372-a567-0e02b2c3d481,Para Cetamol
3,f47ac10b-58cc-4372-a567-0e02b2c3d482,John Doe
4,f47ac10b-58cc-4372-a567-0e02b2c3d483,John Doe
5,f47ac10b-58cc-4372-a567-0e02b2c3d484,John Krasinski
6,f47ac10b-58cc-4372-a567-0e02b2c3d485,John Doe
7,f47ac10b-58cc-4372-a567-0e02b2c3d486,Jennifer Lopez
8,f47ac10b-58cc-4372-a567-0e02b2c3d487,Liav Ichenbaum
9,f47ac10b-58cc-4372-a567-0e02b2c3d488,Curtis Jackson


In [8]:
class TestContactFullNameExtraction(unittest.TestCase):

    def test_valid_contact_data(self):
        """Test extraction with valid contact data."""
        contact_data_str = '[{ "contact_name":"Curtis", "contact_surname":"Jackson", "city":"Chicago", "cp": "12345"}]'
        expected = 'Curtis Jackson'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

    def test_missing_contact_name(self):
        """Test extraction when contact_name is missing."""
        contact_data_str = '[{ "contact_surname":"Jackson", "city":"Chicago", "cp": "12345"}]'
        expected = 'Jackson'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

    def test_missing_contact_surname(self):
        """Test extraction when contact_surname is missing."""
        contact_data_str = '[{ "contact_name":"Curtis", "city":"Chicago", "cp": "12345"}]'
        expected = 'Curtis'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

    def test_missing_both_names(self):
        """Test extraction when both contact_name and contact_surname are missing."""
        contact_data_str = '[{ "city":"Chicago", "cp": "12345"}]'
        expected = 'John Doe'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

    def test_empty_contact_data(self):
        """Test extraction when contact_data is empty."""
        contact_data_str = ''
        expected = 'John Doe'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

    def test_none_contact_data(self):
        """Test extraction when contact_data is None."""
        contact_data_str = None
        expected = 'John Doe'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

    def test_invalid_json(self):
        """Test extraction when contact_data is invalid JSON."""
        contact_data_str = 'invalid json'
        expected = 'John Doe'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

    def test_single_contact_data_dict(self):
        """Test extraction when contact_data is a single dict."""
        contact_data_str = '{ "contact_name":"Liav", "contact_surname": "Ichenbaum", "city":"Tel Aviv"}'
        expected = 'Liav Ichenbaum'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

    def test_contact_data_with_extra_whitespace(self):
        """Test extraction when contact_data has extra whitespace."""
        contact_data_str = ' [ { "contact_name" : "Curtis" , "contact_surname" : "Jackson" } ] '
        expected = 'Curtis Jackson'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

    def test_contact_data_with_quotes(self):
        """Test extraction when contact_data uses single quotes."""
        contact_data_str = "[{ 'contact_name':'Curtis', 'contact_surname':'Jackson'}]"
        expected = 'Curtis Jackson'
        result = extract_contact_full_name(contact_data_str)
        self.assertEqual(result, expected)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

................
----------------------------------------------------------------------
Ran 16 tests in 0.007s

OK


Test 3: DataFrame of Orders with Contact Address

In [9]:
def extract_contact_address(contact_data_str):
    """
    Extracts the contact address from the contact_data field.

    Args:
        contact_data_str (str): A string representing contact data in JSON format.

    Returns:
        str: The contact address in the format "city name, postal code",
             using placeholders if necessary.
    """
    city = 'Unknown'
    postal_code = 'UNK00'

    if not contact_data_str or contact_data_str.strip() == '':
        return f"{city}, {postal_code}"

    try:
        # Replace single quotes with double quotes to handle incorrect JSON
        contact_data_str = contact_data_str.replace("'", '"')
        contact_data = json.loads(contact_data_str)

        # If contact_data is a list, take the first element
        if isinstance(contact_data, list) and contact_data:
            contact_data = contact_data[0]
        elif isinstance(contact_data, dict):
            pass  # contact_data is already a dict
        else:
            return f"{city}, {postal_code}"

        # Extract 'city' and 'cp'
        city = contact_data.get('city', 'Unknown') or 'Unknown'
        postal_code = contact_data.get('cp', 'UNK00') or 'UNK00'

        # Ensure postal_code is a string
        postal_code = str(postal_code)

        return f"{city}, {postal_code}"

    except json.JSONDecodeError:
        # If JSON decoding fails, return placeholders
        return f"{city}, {postal_code}"

def create_contact_address_dataframe(orders):
    """
    Creates a DataFrame with order_id and contact_address columns.

    Args:
        orders (list of dict): List of orders from orders.csv.

    Returns:
        pandas.DataFrame: DataFrame with 'order_id' and 'contact_address' columns.
    """
    data = []

    for order in orders:
        order_id = order.get('order_id')
        contact_data_str = order.get('contact_data')
        contact_address = extract_contact_address(contact_data_str)
        data.append({'order_id': order_id, 'contact_address': contact_address})

    df = pd.DataFrame(data)
    return df

if __name__ == "__main__":
    # Read orders from 'orders.csv'
    orders = read_orders_from_csv('data/orders.csv')

    # Create the DataFrame with contact addresses
    df_2 = create_contact_address_dataframe(orders)

    # Output the DataFrame
    display(df_2)

,order_id,contact_address
0,f47ac10b-58cc-4372-a567-0e02b2c3d479,"Chicago, 12345"
1,f47ac10b-58cc-4372-a567-0e02b2c3d480,"Calcutta, UNK00"
2,f47ac10b-58cc-4372-a567-0e02b2c3d481,"Frankfurt am Oder, 3934"
3,f47ac10b-58cc-4372-a567-0e02b2c3d482,"Unknown, UNK00"
4,f47ac10b-58cc-4372-a567-0e02b2c3d483,"Unknown, UNK00"
5,f47ac10b-58cc-4372-a567-0e02b2c3d484,"New York, 1203"
6,f47ac10b-58cc-4372-a567-0e02b2c3d485,"Unknown, UNK00"
7,f47ac10b-58cc-4372-a567-0e02b2c3d486,"Esplugues de Llobregat, UNK00"
8,f47ac10b-58cc-4372-a567-0e02b2c3d487,"Tel Aviv, UNK00"
9,f47ac10b-58cc-4372-a567-0e02b2c3d488,"Chicago, 12345"


In [10]:
class TestContactAddressExtraction(unittest.TestCase):

    def test_valid_contact_data(self):
        """Test extraction with valid contact data."""
        contact_data_str = '[{ "contact_name":"Curtis", "contact_surname":"Jackson", "city":"Chicago", "cp": "12345"}]'
        expected = 'Chicago, 12345'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

    def test_missing_city(self):
        """Test extraction when city is missing."""
        contact_data_str = '[{ "contact_name":"Curtis", "contact_surname":"Jackson", "cp": "12345"}]'
        expected = 'Unknown, 12345'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

    def test_missing_cp(self):
        """Test extraction when cp is missing."""
        contact_data_str = '[{ "contact_name":"Curtis", "contact_surname":"Jackson", "city": "Chicago"}]'
        expected = 'Chicago, UNK00'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

    def test_missing_both_city_and_cp(self):
        """Test extraction when both city and cp are missing."""
        contact_data_str = '[{ "contact_name":"Curtis", "contact_surname":"Jackson"}]'
        expected = 'Unknown, UNK00'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

    def test_empty_contact_data(self):
        """Test extraction when contact_data is empty."""
        contact_data_str = ''
        expected = 'Unknown, UNK00'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

    def test_none_contact_data(self):
        """Test extraction when contact_data is None."""
        contact_data_str = None
        expected = 'Unknown, UNK00'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

    def test_invalid_json(self):
        """Test extraction when contact_data is invalid JSON."""
        contact_data_str = 'invalid json'
        expected = 'Unknown, UNK00'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

    def test_single_contact_data_dict(self):
        """Test extraction when contact_data is a single dict."""
        contact_data_str = '{ "contact_name":"Liav", "contact_surname": "Ichenbaum", "city":"Tel Aviv"}'
        expected = 'Tel Aviv, UNK00'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

    def test_contact_data_with_numeric_cp(self):
        """Test extraction when cp is numeric."""
        contact_data_str = '[{ "city":"Frankfurt am Oder", "cp": 3934}]'
        expected = 'Frankfurt am Oder, 3934'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

    def test_contact_data_with_extra_whitespace(self):
        """Test extraction when contact_data has extra whitespace."""
        contact_data_str = ' [ { "city" : "Chicago" , "cp" : "12345" } ] '
        expected = 'Chicago, 12345'
        result = extract_contact_address(contact_data_str)
        self.assertEqual(result, expected)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........................
----------------------------------------------------------------------
Ran 26 tests in 0.011s

OK


Test 4: Calculation of Sales Team Commissions

In [11]:
# Function to calculate net value based on gross value and VAT percentage
# VAT is divided by 100 to convert percentage to decimal, and the net value is calculated
def calculate_net_value(gross_value, vat):
    """
    Calculates the net value from gross value and VAT.

    Args:
        gross_value (int): Gross value in cents.
        vat (float): VAT percentage.

    Returns:
        float: Net value in cents.
    """
    net_value = gross_value / (1 + vat / 100)
    return net_value

def calculate_commissions(orders, invoices):
    """
    Calculates the commissions for each sales owner.

    Args:
        orders (list of dict): List of orders.
        invoices (dict): Dictionary of invoices keyed by orderId.

    Returns:
        pandas.DataFrame: DataFrame of sales owners and their total commissions in euros, sorted in descending order.
    """
    commissions = defaultdict(float)

    for order in orders:
        order_id = order.get('order_id')
        salesowners_str = order.get('salesowners')
        invoice = invoices.get(order_id)

        # Skip if invoice is missing or salesowners is empty
        if not invoice or not salesowners_str:
            continue

        # Extract gross_value and vat
        gross_value_cents = int(invoice['grossValue'])
        vat = float(invoice['vat'])
        gross_value_euros = gross_value_cents / 100  # Convert to euros

        # Calculate net_value in euros
        net_value = gross_value_euros / (1 + vat / 100)

        # Extract salesowners and clean names
        salesowners = [owner.strip() for owner in salesowners_str.split(',') if owner.strip()]

        # Assign commissions
        commission_rates = [0.06, 0.025, 0.0095]  # Rates for main owner, co-owner 1, co-owner 2
        for idx, owner in enumerate(salesowners):
            if idx >= len(commission_rates):
                break  # Rest of the co-owners do not receive anything
            commission = net_value * commission_rates[idx]
            # Aggregate commission per owner
            commissions[owner] += commission

    # Round commissions to two decimal places
    commissions_euros = {owner: round(amount, 2) for owner, amount in commissions.items()}

    # Create DataFrame from commissions
    df_commissions = pd.DataFrame(list(commissions_euros.items()), columns=['Sales Owner', 'Total Commission (€)'])
    # Sort the DataFrame by total commissions in descending order
    df_commissions = df_commissions.sort_values(by='Total Commission (€)', ascending=False).reset_index(drop=True)

    return df_commissions


if __name__ == "__main__":
    # Read data from 'orders.csv' and 'invoicing_data.json'
    orders = read_orders_from_csv('data/orders.csv')
    invoices = read_invoices_from_json('data/invoicing_data.json')

    # Calculate commissions
    df_commissions = calculate_commissions(orders, invoices)

    # Output the commissions DataFrame
    print("Sales Owner Commissions:")
    display(df_commissions)

Sales Owner Commissions:


,Sales Owner,Total Commission (€)
0,Leonard Cohen,650.31
1,David Henderson,463.29
2,Luke Skywalker,377.61
3,David Goliat,214.96
4,Ammy Winehouse,209.52
5,Marianov Merschik,188.61
6,Yuri Gagarin,154.62
7,Chris Pratt,114.08
8,Vladimir Chukov,72.83
9,Marie Curie,70.52


In [12]:
class TestCommissionCalculations(unittest.TestCase):

    def setUp(self):
        # Sample orders
        self.orders = [
            {
                'order_id': 'order1',
                'salesowners': 'Alice, Bob, Charlie'
            },
            {
                'order_id': 'order2',
                'salesowners': 'Bob, Alice'
            },
            {
                'order_id': 'order3',
                'salesowners': 'Charlie'
            },
            {
                'order_id': 'order4',
                'salesowners': 'Dana'
            },
            {
                'order_id': 'order5',
                'salesowners': ''
            }
        ]

        # Sample invoices
        self.invoices = {
            'order1': {'grossValue': '119000', 'vat': '19'},
            'order2': {'grossValue': '238000', 'vat': '19'},
            'order3': {'grossValue': '100000', 'vat': '0'},
            'order4': {'grossValue': '121000', 'vat': '21'},
            'order5': {'grossValue': '100000', 'vat': '19'}
        }

# Unit test to validate the correct calculation of net values from gross values
    def test_calculate_net_value(self):
        # Test net value calculation
        gross_value = 119000
        vat = 19
        expected_net = 1000.00  # In euros
        net_value = calculate_net_value(gross_value, vat) / 100  # Convert to euros
        self.assertAlmostEqual(net_value, expected_net)

    def test_calculate_commissions(self):
        # Expected commissions calculated with correct rates and net values in euros
        expected_commissions = [
            {'Sales Owner': 'Bob', 'Total Commission (€)': 145.0},
            {'Sales Owner': 'Alice', 'Total Commission (€)': 110.0},
            {'Sales Owner': 'Charlie', 'Total Commission (€)': 69.5},
            {'Sales Owner': 'Dana', 'Total Commission (€)': 60.0}
        ]
        expected_df = pd.DataFrame(expected_commissions)
        expected_df = expected_df.sort_values(by='Sales Owner').reset_index(drop=True)

        commissions_df = calculate_commissions(self.orders, self.invoices)
        commissions_df = commissions_df.sort_values(by='Sales Owner').reset_index(drop=True)

        pdt.assert_frame_equal(commissions_df, expected_df)

    def test_missing_invoice(self):
        # Test handling of missing invoices
        orders = [{'order_id': 'order_missing', 'salesowners': 'Alice'}]
        invoices = {}
        commissions_df = calculate_commissions(orders, invoices)
        expected_df = pd.DataFrame(columns=['Sales Owner', 'Total Commission (€)'])
        pdt.assert_frame_equal(commissions_df, expected_df)

    def test_missing_salesowners(self):
        # Test handling of orders without salesowners
        orders = [{'order_id': 'order1', 'salesowners': None}]
        invoices = {'order1': {'grossValue': '100000', 'vat': '0'}}
        commissions_df = calculate_commissions(orders, invoices)
        expected_df = pd.DataFrame(columns=['Sales Owner', 'Total Commission (€)'])
        pdt.assert_frame_equal(commissions_df, expected_df)

    def test_commission_calculation_accuracy(self):
        # Test commission calculation with specific values
        orders = [{'order_id': 'order1', 'salesowners': 'Alice, Bob, Charlie'}]
        invoices = {'order1': {'grossValue': '119000', 'vat': '19'}}
        # Expected commissions
        expected_commissions = [
            {'Sales Owner': 'Alice', 'Total Commission (€)': 60.0},
            {'Sales Owner': 'Bob', 'Total Commission (€)': 25.0},
            {'Sales Owner': 'Charlie', 'Total Commission (€)': 9.5}
        ]
        expected_df = pd.DataFrame(expected_commissions)
        expected_df = expected_df.sort_values(by='Sales Owner').reset_index(drop=True)

        commissions_df = calculate_commissions(orders, invoices)
        commissions_df = commissions_df.sort_values(by='Sales Owner').reset_index(drop=True)

        pdt.assert_frame_equal(commissions_df, expected_df)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...............................
----------------------------------------------------------------------
Ran 31 tests in 0.017s

OK


Test 5: DataFrame of Companies with Sales Owners

In [13]:
def normalize_company_name(company_name): 
    """
    Normalize company names by converting to lowercase and removing common suffixes.

    Args:
        company_name (str): Original company name.

    Returns:
        str: Normalized company name.
    """
    # Convert to lowercase
    name = company_name.lower()
    # Remove common suffixes
    suffixes = [' co', ' co.', ' c.o.', ' inc', ' inc.', ' ltd', ' ltd.', ' gmbh']
    for suffix in suffixes:
        if name.endswith(suffix):
            name = name[:-len(suffix)]
            break
    # Remove any extra whitespace
    name = name.strip()
    return name

def create_company_salesowners_dataframe(orders):
    """
    Create a DataFrame with unique companies and their associated salesowners.

    Args:
        orders (list of dict): List of orders.

    Returns:
        pandas.DataFrame: DataFrame containing company details and salesowners.
    """
    company_dict = {}

    for order in orders:
        company_id = order.get('company_id')
        company_name = order.get('company_name')
        salesowners = order.get('salesowners', '')

        # Normalize the company name
        normalized_name = normalize_company_name(company_name)

        # If the normalized company is not in the dictionary, add it
        if normalized_name not in company_dict:
            company_dict[normalized_name] = {
                'company_id': company_id,
                'company_name': company_name,
                'salesowners_set': set()
            }
        else:
            # Keep the smallest company_id
            if company_id < company_dict[normalized_name]['company_id']:
                company_dict[normalized_name]['company_id'] = company_id
                company_dict[normalized_name]['company_name'] = company_name

        # Add salesowners to the set
        if salesowners:
            owners = [owner.strip() for owner in salesowners.split(',') if owner.strip()]
            company_dict[normalized_name]['salesowners_set'].update(owners)

    # Prepare data for DataFrame
    data = []
    for company in company_dict.values():
        # Sort salesowners alphabetically and join them into a string
        salesowners_list = sorted(company['salesowners_set'])
        salesowners_str = ', '.join(salesowners_list)
        data.append({
            'company_id': company['company_id'],
            'company_name': company['company_name'],
            'list_salesowners': salesowners_str
        })

    # Create DataFrame
    df = pd.DataFrame(data)
    return df

if __name__ == "__main__":
    # Read orders from 'orders.csv'
    orders = read_orders_from_csv('data/orders.csv')

    # Create the DataFrame with company salesowners
    df_3 = create_company_salesowners_dataframe(orders)

    # Output the DataFrame
    display(df_3)

,company_id,company_name,list_salesowners
0,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits Co,"Ammy Winehouse, Leonard Cohen, Luke Skywalker"
1,0f05a8f1-2bdf-4be7-8c82-4c9b58f04898,Veggies Inc,"David Goliat, Leon Leonov, Luke Skywalker"
2,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits c.o,Luke Skywalker
3,1c4b0b50-1d5d-463a-b56e-1a6fd3aeb7d6,Seafood Supplier,"Ammy Winehouse, David Goliat, Leonard Cohen, Markus Söder"
4,34538e39-cd2e-4641-8d24-3c94146e6f16,Meat Packers Ltd,"Chris Pratt, David Henderson, Leon Leonov, Marianov Merschik"
5,fa14c3ed-3c48-49f4-bd69-4d7f5b5f4b1b,Green Veg Co,"David Henderson, Leonard Cohen"
6,acdb6f30-764f-404e-8b8e-7e7e3e6fa1a9,Organic Farms,"David Henderson, Leon Leonov, Leonard Cohen"
7,5f0bdbdf-1d84-4c23-957c-8bb8c0ddc89d,Tropical Fruits Ltd,"David Goliat, David Henderson, Yuri Gagarin"
8,20dfef10-8f4e-45a1-82fc-123f4ab2a4a5,healthy snacks c.o.,"Ammy Winehouse, Chris Pratt, Luke Skywalker, Marianov Merschik, Marie Curie, Vladimir Chukov"


In [14]:
class TestCompanySalesownersDataframe(unittest.TestCase):

    def setUp(self):
        self.orders = [
            {
                'order_id': 'order1',
                'company_id': 'id1',
                'company_name': 'Acme Co',
                'salesowners': 'Alice, Bob'
            },
            {
                'order_id': 'order2',
                'company_id': 'id2',
                'company_name': 'Acme Co.',
                'salesowners': 'Charlie'
            },
            {
                'order_id': 'order3',
                'company_id': 'id3',
                'company_name': 'Beta Inc',
                'salesowners': 'Dana'
            },
            {
                'order_id': 'order4',
                'company_id': 'id4',
                'company_name': 'Gamma Ltd',
                'salesowners': 'Eve, Frank'
            },
            {
                'order_id': 'order5',
                'company_id': 'id5',
                'company_name': 'Acme c.o.',
                'salesowners': 'George, Alice'
            },
            {
                'order_id': 'order6',
                'company_id': 'id6',
                'company_name': 'Gamma ltd',
                'salesowners': 'Heidi'
            }
        ]

    def test_dataframe_creation(self):
        df = create_company_salesowners_dataframe(self.orders)
        # Expected companies after normalization: 'Acme Co', 'Beta Inc', 'Gamma Ltd'
        self.assertEqual(len(df), 3)
        # Check that company names are as expected
        expected_company_names = {'Acme Co', 'Beta Inc', 'Gamma Ltd'}
        self.assertEqual(set(df['company_name']), expected_company_names)
        # Check that salesowners are unique and sorted for 'Acme Co'
        acme_row = df[df['company_name'] == 'Acme Co'].iloc[0]
        self.assertEqual(acme_row['company_id'], 'id1')
        expected_salesowners = 'Alice, Bob, Charlie, George'
        self.assertEqual(acme_row['list_salesowners'], expected_salesowners)

    def test_company_id_selection(self):
        df = create_company_salesowners_dataframe(self.orders)
        # For 'Acme Co', the company_id should be the smallest one ('id1')
        acme_row = df[df['company_name'] == 'Acme Co'].iloc[0]
        self.assertEqual(acme_row['company_id'], 'id1')

    def test_salesowners_sorting(self):
        df = create_company_salesowners_dataframe(self.orders)
        gamma_row = df[df['company_name'] == 'Gamma Ltd'].iloc[0]
        expected_salesowners = 'Eve, Frank, Heidi'
        self.assertEqual(gamma_row['list_salesowners'], expected_salesowners)

# Unit test to ensure that duplicate company names are handled correctly in the output
    def test_handling_duplicates(self):
        df = create_company_salesowners_dataframe(self.orders)
        # Ensure there are no duplicate company names after processing
        company_names = df['company_name'].tolist()
        self.assertEqual(len(company_names), len(set(company_names)))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...................................
----------------------------------------------------------------------
Ran 35 tests in 0.026s

OK
